In [ ]:
from time import sleep
import streamlit as st
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from function import *

In [47]:
# Récuparation des 2000 premières données
scrap = Scrap()
data = scrap.scrap_immo('France', 2000)

In [48]:
# Création du fichier JSON
import json

with open('offers.json', 'w') as outfile:
    json.dump(data, outfile)
print('Success')

Success


## Création et insertion en base de données

In [49]:
# Gestion d'une base de données
import sqlalchemy as db
from sqlalchemy import or_

class DataBase():
    def __init__(self, name_database='database'):
        self.name = name_database
        self.url = f"sqlite:///{name_database}.db"
        self.engine = db.create_engine(self.url)
        self.connection = self.engine.connect()
        self.metadata = db.MetaData()
        self.table = self.engine.table_names()


    def create_table(self, name_table, **kwargs):
        colums = [db.Column(k, v, primary_key = True) if 'id_' in k else db.Column(k, v) for k,v in kwargs.items()]
        db.Table(name_table, self.metadata, *colums)
        self.metadata.create_all(self.engine)
        print(f"Table : '{name_table}' are created succesfully")

    def read_table(self, name_table, return_keys=False):
        table = db.Table(name_table, self.metadata, autoload=True, autoload_with=self.engine)
        if return_keys:table.columns.keys()
        else : return table


    def add_row(self, name_table, **kwarrgs):
        name_table = self.read_table(name_table)

        stmt = (
            db.insert(name_table).
            values(kwarrgs)
        )
        self.connection.execute(stmt)
        print(f'Row id added')


    def delete_row_by_id(self, table, id_):
        name_table = self.read_table(name_table)

        stmt = (
            db.delete(name_table).
            where(students.c.id_ == id_)
            )
        self.connection.execute(stmt)
        print(f'Row id {id_} deleted')

    def select_table(self, name_table, filter_by=None):
        name_table_ref = self.read_table(name_table)
        query = db.select([name_table_ref])

        if filter_by:
            conditions = [getattr(name_table_ref.c, key).like(f"%{value}%") for key, value in filter_by.items()]
            query = query.where(or_(*conditions))

        return self.connection.execute(query).fetchall()


In [50]:
# Création de la base de donnée et de la table offre
database = DataBase('immoDB')
database.create_table('offres',
                id_=db.Integer,
                link=db.String,
                image=db.String,
                name=db.String,
                place=db.String,
                size=db.Integer,
                price=db.Integer,
                square_price=db.Float
                )

Table : 'offres' are created succesfully


/var/folders/hw/wy1jtnxj1gbgkbvb6zw_8st00000gn/T/ipykernel_16821/2643536071.py:12: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  self.table = self.engine.table_names()


In [51]:
# Insertion des 2000 premières données dans la base de donnée
for url, details in data.items():
    database.add_row('offres', 
                     link=url, 
                     image=details['image'], 
                     name=details['name'], 
                     place=details['place'], 
                     size=details['size'],
                     price=details['price'],
                     square_price=details['square_price']
                     )

Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added
Row id added

In [54]:
# Création de la base de donnée pour la stockage des historiques
database.create_table("historique", id_=db.Integer, ip=db.String, search=db.String)

Table : 'historique' are created succesfully
